In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [4]:
# !pip install --upgrade tensorflow--gpu

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 450kB 65.6MB/s 
     |████████████████████████████████| 3.8MB 68.0MB/s 
     |████████████████████████████████| 81kB 13.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model

In [0]:
HEIGHT = 224
WIDTH = 224

base_model = ResNet50(weights = 'imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

In [0]:
for layer in base_model.layers:
  layer.trainable = False

for layer in base_model.layers:
  layer.trainable = True

In [0]:
x = base_model.output
x = Flatten()(x)

x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.25)(x)
#New Softmax layer
import time
s = time.time()
predictions = Dense(2, activation = 'softmax')(x)
print(time.time()-s)
model = Model(inputs=base_model.input, outputs=predictions)


In [0]:
import zipfile
with zipfile.ZipFile('//content/drive/My Drive/chest_xray.zip', 'r') as zip_ref: zip_ref.extractall('')

In [0]:
train_data_dir = '/content/chest_xray/train'
validation_data_dir = '/content/chest_xray/test'

In [13]:
epoch = 4
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(224, 224), batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir, target_size=(224, 224), batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [0]:
model.compile("adam", loss='categorical_crossentropy', metrics=["accuracy"])

In [0]:
%%time
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.n // batch_size,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 326 steps, validate for 39 steps
Epoch 1/4
326/326 [==============================] - 141s 431ms/step - loss: 7070772678.6748 - accuracy: 0.5253 - val_loss: 2941766524.7179 - val_accuracy: 1.0000
Epoch 2/4
326/326 [==============================] - 135s 415ms/step - loss: 7646044893.4479 - accuracy: 0.5247 - val_loss: 523029523.9744 - val_accuracy: 0.0000e+00
Epoch 3/4
326/326 [==============================] - 136s 418ms/step - loss: 8421569262.7239 - accuracy: 0.5075 - val_loss: 9214186837.3333 - val_accuracy: 1.0000


In [0]:
model.summary()